In [ ]:
pip install google-generativeai langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain==0.3.19
!pip install langchain-google-genai
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Gemini API key
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

''

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(
    model = 'gemini-pro',
    temperature=0)

# Using GeminiAI  
# Task1 : Research paper review - attention all you need (Transformers) or word2vec 2014 paper

# Converting PDF to text format

In [ ]:
!pip install pdfplumber


In [ ]:
import pdfplumber

def pdf_to_text(pdf_path, text_file_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    # save to a text file
    with open(text_file_path, 'w', encoding='utf-8') as f:
        f.write(text)

    return text

pdf_path = "/content/NIPS-2017-attention-is-all-you-need-Paper.pdf"
text_path ='attention_is_all_you_need.txt'
text = pdf_to_text(pdf_path, text_path)
print('PDF converted to text successfully')

PDF converted to text successfully


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


In [ ]:
# Define a data structure
class QueryResponse(BaseModel):
  Key_findings : str = Field(description="key findings of the model")
  pros: str = Field(description="pros of the model")
  cons: str = Field(description="cons of the model")

parser = PydanticOutputParser(pydantic_object=QueryResponse)
parser

PydanticOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [ ]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Key_findings": {"description": "key findings of the model", "title": "Key Findings", "type": "string"}, "pros": {"description": "pros of the model", "title": "Pros", "type": "string"}, "cons": {"description": "cons of the model", "title": "Cons", "type": "string"}}, "required": ["Key_findings", "pros", "cons"]}
```


In [ ]:
prompt_text ='''act like a professional, explain in detail with 3 bullet points of
the key findings, pros and cons of the model in simple english
 Format Instruction: {format_instructions}
 Text:{text}
 '''
prompt = PromptTemplate(
    template=prompt_text,
    input_variables=['text'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm , prompt=prompt)

# load the extracted text from the file
with open(text_path, 'r') as f:
    text = f.read()
response = chain.invoke({'text':text})

response



{'text': '```json\n{\n "properties": {\n  "Key_findings": "The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolutions.",\n  "pros": "- The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.\\n- The Transformer achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU.\\n- On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature.",\n  "cons": "- The Transformer is more computationally expensive than recurrent or convolutional layers when the sequenc

In [ ]:
for k, v in response.items():
  print(f"{k}: \n{v}\n")

text: 
```json
{
 "properties": {
  "Key_findings": "The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolutions.",
  "pros": "- The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.\n- The Transformer achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU.\n- On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature.",
  "cons": "- The Transformer is more computationally expensive than recurrent or convolutional layers when the sequence length i

Summary of the same

In [ ]:
from langchain.chains import LLMChain
prompt1 = PromptTemplate(template = """Summarization of paper 'Attention is all you need'
    in simple terms. Focus on key concepts like self-attention, transformer architecture,
    and benefits over RNNs""",
                         input_variables = [])
chain = LLMChain(llm=llm, prompt=prompt1)
summary = chain.invoke({})
print(summary)

<ipython-input-9-8c39879d1728>:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt1)


{'text': '**Attention is all you need**\n\n**Key Concepts:**\n\n* **Self-Attention:** A mechanism that allows a model to focus on specific parts of its own input sequence.\n* **Transformer Architecture:** A neural network architecture that uses self-attention instead of recurrent neural networks (RNNs).\n\n**Benefits over RNNs:**\n\n* **Parallel Processing:** Transformers can process all input tokens simultaneously, unlike RNNs which process tokens sequentially.\n* **Long-Range Dependencies:** Transformers can capture long-range dependencies in the input sequence, which RNNs struggle with.\n* **Scalability:** Transformers can be scaled up to larger datasets and longer sequences than RNNs.\n\n**How it Works:**\n\n1. **Input Embedding:** The input sequence is converted into a series of vectors.\n2. **Self-Attention:** Each vector is compared to all other vectors in the sequence, creating a matrix of attention weights.\n3. **Weighted Sum:** The attention weights are used to create a weigh

In [ ]:
for k, v in summary.items():
  print(f'{k} :\n {v}\n')

text :
 **Attention is all you need**

**Key Concepts:**

* **Self-Attention:** A mechanism that allows a model to focus on specific parts of its own input sequence.
* **Transformer Architecture:** A neural network architecture that uses self-attention instead of recurrent neural networks (RNNs).

**Benefits over RNNs:**

* **Parallel Processing:** Transformers can process all input tokens simultaneously, unlike RNNs which process tokens sequentially.
* **Long-Range Dependencies:** Transformers can capture long-range dependencies in the input sequence, which RNNs struggle with.
* **Scalability:** Transformers can be scaled up to larger datasets and longer sequences than RNNs.

**How it Works:**

1. **Input Embedding:** The input sequence is converted into a series of vectors.
2. **Self-Attention:** Each vector is compared to all other vectors in the sequence, creating a matrix of attention weights.
3. **Weighted Sum:** The attention weights are used to create a weighted sum of the inpu

# Task 2: IT Support Query Resolution-
* Translation
* Solution
* Support Email Generation

In [ ]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]


In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class it_support(BaseModel):
  query: str = Field(description="translate the Question to english")
  solution: str = Field(description="Solution to the question")
  email: str = Field(description = "Detailed email to the customer answering the query")

parser = PydanticOutputParser(pydantic_object=it_support)
parser

PydanticOutputParser(pydantic_object=<class '__main__.it_support'>)

In [ ]:
prompt_text1 = """
Analyse the given customer query and generate the response based on the instructions metioned
below in the format instructions
Also write a detailed email response in email field based on these
-email should be addressed to Dear Customer and signed as server agent
- thank them in the end
-apologize if there is negative query
-write the email in respected language as in the query

Format Instructions:
{format_instructions}

Customer Query:
{it_support_queue}
"""
prompt = PromptTemplate(
    template=prompt_text1,
    input_variables=['it_support_queue'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['it_support_queue'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"query": {"description": "translate the Question to english", "title": "Query", "type": "string"}, "solution": {"description": "Solution to the question", "title": "Solution", "type": "string"}, "email": {"description": "Detailed email to the customer answering the query", "title": "Email", "type": "string"}}, "required": ["query", "solution", "email"]}\n```'}, template='\nAnalyse the given customer query and g

In [ ]:
#Format the IT_Support-queue
queue = [{'it_support_queue':i} for i in it_support_queue ]
queue

[{'it_support_queue': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.'},
 {'it_support_queue': 'Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.'},
 {'it_support_queue': 'プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。'},
 {'it_support_queue': 'Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.'},
 {'it_support_queue': 'Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?'},
 {'it_support_queue': 'Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста.'}]

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.map().invoke(queue)
response[0]

{'it_support_queue': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
 'text': '```\n{\n  "query": "I am unable to sync my contacts to my phone. I keep getting a failed message.",\n  "solution": "**Troubleshooting Contact Syncing Issues**\\n\\n**1. Check your internet connection.** Make sure you have a strong and stable internet connection. A weak or intermittent connection can cause syncing issues.\\n\\n**2. Restart your phone and the Google Contacts app.** This can help to clear any temporary glitches that may be preventing syncing.\\n\\n**3. Check your Google Account settings.** Make sure that syncing is enabled for your Google Account. You can check this by going to the Google Account settings page and selecting the \\"Data & personalization\\" tab. Under \\"Activity controls,\\" make sure that the \\"Contacts\\" toggle is turned on.\\n\\n**4. Check your phone\'s settings.** Make sure that syncing is enabled for the Google Contacts app. 

In [ ]:
for i in range(len(response)):
  for k, v in response[i].items():
    print(f'{k} : \n{v}\n')
    print('**************')
    print('\n')

it_support_queue : 
Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.

**************


text : 
```
{
  "query": "I am unable to sync my contacts to my phone. I keep getting a failed message.",
  "solution": "**Troubleshooting Contact Syncing Issues**\n\n**1. Check your internet connection.** Make sure you have a strong and stable internet connection. A weak or intermittent connection can cause syncing issues.\n\n**2. Restart your phone and the Google Contacts app.** This can help to clear any temporary glitches that may be preventing syncing.\n\n**3. Check your Google Account settings.** Make sure that syncing is enabled for your Google Account. You can check this by going to the Google Account settings page and selecting the \"Data & personalization\" tab. Under \"Activity controls,\" make sure that the \"Contacts\" toggle is turned on.\n\n**4. Check your phone's settings.** Make sure that syncing is enabled for the Google Contacts app. You ca